#**Pre-request**

##Mount google drive


In [1]:
### **Mount** Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


##Install pakages


In [ ]:
#Install pakages
project_path = "/content/drive/MyDrive/Sem-6/coding/github/fraud_detection/"
!cat "{project_path}requirement/Install/EnhancedPretraindMLModleAdvance.txt"
!pip install  -r "{project_path}requirement/Install/EnhancedPretraindMLModleAdvance.txt" --no-cache-dir
%cd $project_path





##Import  libs

In [ ]:
# =====================================================
# 📦 Standard Library Imports
# =====================================================
import os
import yaml
import logging
import datetime
import sys


# =====================================================
# 🧮 Data & Visualization
# =====================================================
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from google.colab import data_table
data_table.enable_dataframe_formatter()

# Expand Colab’s table display limits
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)

# =====================================================
# ⚙️ Machine Learning - Scikit-learn
# =====================================================
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, RobustScaler, StandardScaler
from sklearn.utils import class_weight
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

# Covariance for Mahalanobis distance
from sklearn.covariance import EmpiricalCovariance

# Evaluation metrics (all consolidated)
from sklearn.metrics import (
    roc_auc_score,
    average_precision_score,
    precision_score,
    recall_score,
    f1_score,
    roc_curve,
    precision_recall_curve,
    classification_report,
    confusion_matrix,
    ConfusionMatrixDisplay
)

# =====================================================
# 🤖 Deep Learning - TensorFlow / Keras
# =====================================================
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Masking, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K

# =====================================================
# 🔥 Deep Learning - PyTorch
# =====================================================
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
from torch.cuda.amp import autocast

# =====================================================
# 🤗 Transformers & Advanced Models
# =====================================================
from transformers import AutoModel
# from mamba_ssm import Mamba  # Uncomment if used

# =====================================================
# 🧠 Explainability
# =====================================================
import shap

##Confirmation setup

In [ ]:
!nvidia-smi                # confirm GPU
!pip show torch  # confirm versions
torch.manual_seed(42)


##Config and Var

In [ ]:

logger = logging.getLogger(__name__)

def load_config(config_path="configs/baseline.yaml"):
    """Load YAML config file and expand ${root_path} placeholders."""
    with open(config_path, "r") as f:
        config = yaml.safe_load(f)

    logger.info(f"✅ Loaded config from {config_path}")

    # --- Expand ${root_path} placeholders ---
    root = config.get("root_path", "")

    def expand_paths(obj):
        if isinstance(obj, dict):
            return {k: expand_paths(v) for k, v in obj.items()}
        elif isinstance(obj, list):
            return [expand_paths(i) for i in obj]
        elif isinstance(obj, str) and "${root_path}" in obj:
            return obj.replace("${root_path}", root)
        else:
            return obj

    config = expand_paths(config)
    return config
config = load_config(os.path.join(project_path, "configs", "baseline.yaml"))


limit = config['ML']['limit']
max_seq_len = limit
time_mode='time2vec'
epochs = 5
batch_size=64
correlation_threshold = 0.85
threshold = 0.5
dropout= 0.0




##Split users level

In [ ]:

user_path = config['ML']['Events']['base_path'] + config['ML']['Events']['files']['user']
df_user = pd.read_csv(user_path)
print(f"✅ Loaded transactional user dataset: {df_user.shape}")



# Aggregate to one row per user (max label = 1 if any fraud)
user_labels = df_user.groupby("phone_no_m")["label"].max()
print(f"👥 Unique users for splitting: {len(user_labels)}")

# ==============================================================
# 2️⃣ Create user-level split (stratified, no leakage)
# ==============================================================

fraud_users = user_labels[user_labels == 1].index
normal_users = user_labels[user_labels == 0].index

fraud_train, fraud_test = train_test_split(fraud_users, test_size=0.2, random_state=42)
normal_train, normal_test = train_test_split(normal_users, test_size=0.2, random_state=42)

train_users = set(fraud_train) | set(normal_train)
test_users  = set(fraud_test)  | set(normal_test)

# ==============================================================
# 3️⃣ Save unified split (shared across LSTM / RF / XGB)
# ==============================================================

split_dir = "splits"
os.makedirs(split_dir, exist_ok=True)

pd.DataFrame({"phone_no_m": sorted(train_users)}).to_csv(f"{split_dir}/train_users.csv", index=False)
pd.DataFrame({"phone_no_m": sorted(test_users)}).to_csv(f"{split_dir}/test_users.csv", index=False)

# ==============================================================
# 4️⃣ Summary
# ==============================================================

print("\n👥 Users Summary:")
print(f"   Total : {len(user_labels):,}")
print(f"   Fraud : {len(fraud_users):,} ({len(fraud_users)/len(user_labels)*100:.2f}%)")
print(f"   Normal: {len(normal_users):,} ({len(normal_users)/len(user_labels)*100:.2f}%)")

print("\n📂 Split saved to /splits/:")
print(f"   Train users: {len(train_users)}")
print(f"   Test  users: {len(test_users)}")
print(f"   Fraud ratio train: {len(fraud_train)/len(train_users)*100:.2f}%")
print(f"   Fraud ratio test : {len(fraud_test)/len(test_users)*100:.2f}%")


####Helpers

###Evaluate

In [ ]:
def evaluate_global(model, X_test, y_test, model_name="Model"):
    """
    Generic evaluator for both classic ML models and neural networks (like LSTM).
    Automatically detects the correct prediction method.
    """
    # ---- Predict probabilities ----
    if hasattr(model, "predict_proba"):
        # For sklearn-style models
        y_pred_prob = model.predict_proba(X_test)[:, 1]
    else:
        # For neural nets (e.g., Keras)
        y_pred_prob = model.predict(X_test).ravel()

    # ---- Predict classes ----
    y_pred = (y_pred_prob > threshold).astype(int)

    # ---- Metrics ----
    auc = roc_auc_score(y_test, y_pred_prob)
    recall = recall_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, zero_division=0)
    f1 = f1_score(y_test, y_pred)
    report = classification_report(y_test, y_pred, digits=4)
    cm = confusion_matrix(y_test, y_pred)

    # ---- Display ----
    print(f"\n📊 Classification Report — {model_name}")
    print(report)
    print(f"AUC: {auc:.4f} | Recall: {recall:.4f} | Precision: {precision:.4f} | F1: {f1:.4f}")

    # ---- Confusion Matrix ----
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["Normal (0)", "Fraud (1)"])
    disp.plot(cmap="Blues")
    plt.title(f"Confusion Matrix — {model_name}")
    plt.grid(False)
    plt.show()

    # ---- Summary Dictionary ----
    return {
        "Model": model_name,
        "AUC": auc,
        "Recall": recall,
        "Precision": precision,
        "F1": f1
    }


#ML Modules

##Classic Ml Snapshot based

###Load snapshots

In [ ]:


snapshot_path = config['ML']['snapshot_input'] + config['ML']['snapshot_file']
df = pd.read_csv(snapshot_path)
max_snapshot = df["snapshot_index"].max()


print(f"Input data Max snapshot_index is : {max_snapshot}")

# Filter rows where snapshot_index <= 100
df = df[df["snapshot_index"] <= limit]
max_snapshot = df["snapshot_index"].max()

# Print the maximum snapshot_index before or equal to 100
print(f"Filtred Data Max snapshot_index is : {max_snapshot}")



print(f"✅ Loaded snapshot dataset: {df.shape}")

display((df.head()))

###Drop and select features

In [ ]:
def prepare_features(df):
    """
    Selects only the explicitly defined features for model training.
    You control which features are used by editing 'selected_features' below.
    """

    # --- Define selected features manually ---
    selected_features = [
        "window_size", "voc_total_calls", "voc_unique_contacts", "voc_total_duration",
       "voc_avg_duration", "voc_max_duration", "voc_std_duration", "voc_active_days",
       "voc_active_hours", "sms_total_msgs", "sms_unique_contacts", "sms_active_hours",
       "sms_calltype_ratio", "app_months_active", "app_total_flow", "app_avg_flow",
       "app_std_flow", "app_unique_apps_mean", "app_unique_apps_max", "user_months_active",
        "arpu_mean", "arpu_std", "arpu_max", "idcard_cnt", "snapshot_round"
   ]
  #  selected_features = [
   #     "voc_total_calls", "voc_unique_contacts", "voc_total_duration",
    #   "voc_avg_duration", "voc_max_duration", "voc_std_duration", "voc_active_days",
     # "voc_active_hours", "sms_total_msgs", "sms_unique_contacts", "sms_active_hours",
     #"sms_calltype_ratio", "idcard_cnt"
    #]
   # selected_features = [
    #    "voc_active_days",
    #"voc_active_hours",
    #"voc_unique_contacts",
    #"sms_calltype_ratio",
    #"sms_active_hours" ]


    # ✅ You can manually remove or comment out features here
    # For example:
    # selected_features = [f for f in selected_features if not (f.startswith("app_") or f.startswith("arpu_"))]

    # --- Keep only existing columns ---
    available = [f for f in selected_features if f in df.columns]
    missing = [f for f in selected_features if f not in df.columns]

    X = df[available].copy()

    #print(f"\n📊 Final features used ({len(available)}): {available}")
    if missing:
        print(f"⚠️ Missing columns not found in data: {missing}")

    return X


###Split for snapshot based

In [ ]:


# ==============================================================
# 1️⃣ Load or Create Unified User Split
# ==============================================================

split_dir = "splits"
train_split_file = f"{split_dir}/train_users.csv"
test_split_file  = f"{split_dir}/test_users.csv"

if os.path.exists(train_split_file) and os.path.exists(test_split_file):
    print("📂 Using existing user split from file...")
    train_users = set(pd.read_csv(train_split_file)["phone_no_m"])
    test_users  = set(pd.read_csv(test_split_file)["phone_no_m"])
else:
    print("🆕 Creating new unified user split...")
    os.makedirs(split_dir, exist_ok=True)

    user_labels = df.groupby("phone_no_m")["label"].max()
    fraud_users  = user_labels[user_labels == 1].index
    normal_users = user_labels[user_labels == 0].index

    fraud_train, fraud_test = train_test_split(fraud_users, test_size=0.2, random_state=42)
    normal_train, normal_test = train_test_split(normal_users, test_size=0.2, random_state=42)

    train_users = set(fraud_train) | set(normal_train)
    test_users  = set(fraud_test)  | set(normal_test)

    pd.DataFrame({"phone_no_m": sorted(train_users)}).to_csv(train_split_file, index=False)
    pd.DataFrame({"phone_no_m": sorted(test_users)}).to_csv(test_split_file, index=False)
    print(f"✅ Saved user split to '{split_dir}/'")

print(f"✅ Train users: {len(train_users)} | Test users: {len(test_users)}")

# ==============================================================
# 2️⃣ Apply User Split to Snapshot Data
# ==============================================================

train_df = df[df["phone_no_m"].isin(train_users)]
test_df  = df[df["phone_no_m"].isin(test_users)]

assert len(set(train_df["phone_no_m"]) & set(test_df["phone_no_m"])) == 0, "❌ User leakage detected!"
assert train_df["label"].nunique() == 2, "❌ Training set must contain both classes"
assert test_df["label"].nunique() == 2, "❌ Test set must contain both classes"

print(f"\n👥 User Summary:")
print(f"   Train users: {len(train_users):,}")
print(f"   Test  users: {len(test_users):,}")

print(f"\n📊 Event Split Summary:")
for name, df_part in [("Train", train_df), ("Test", test_df)]:
    total = len(df_part)
    fraud = df_part["label"].sum()
    print(f"   {name:5s} → {total:,} snapshots | Fraud: {fraud/total*100:.2f}% | Normal: {(1 - fraud/total)*100:.2f}%")

# ==============================================================
# 3️⃣ Feature Preparation
# ==============================================================



# Case 1: include app* and arpu* columns
X_train = prepare_features(train_df)
X_test  = prepare_features(test_df)


# Case 2: exclude app* and arpu* columns
# X_train = prepare_features(train_df, use_app_arpu=False)
# X_test  = prepare_features(test_df, use_app_arpu=False)


# Align features (ensure same structure)
X_train, X_test = X_train.align(X_test, join="left", axis=1, fill_value=0)

y_train = train_df["label"].astype(int)
y_test  = test_df["label"].astype(int)

# ==============================================================
# 4️⃣ Full Dataset Scan + Scaling
# ==============================================================

# Combine all snapshots (for robust scaling reference)
all_data = pd.concat([train_df, test_df], axis=0)
print(f"\n📦 Scanning full dataset for scaling — total rows: {len(all_data):,}, columns: {len(all_data.columns)}")

numeric_cols = X_train.columns  # only use columns actually selected for the model
summary = all_data[numeric_cols].describe().T


# Select columns to scale (exclude binary or constants)
scale_cols = [
    c for c in numeric_cols
    if (summary.loc[c, "max"] - summary.loc[c, "min"]) > 5 and summary.loc[c, "max"] > 1
]
scale_cols = [c for c in scale_cols if c in X_train.columns]

print("\n📊 Numeric feature summary (before scaling):")
print(summary[["min", "max", "mean"]].round(2).sort_values("max", ascending=False))

print("\n📏 Selected for scaling (auto-detected based on range):")
print(scale_cols)

# Apply scaling
scaler = RobustScaler().fit(all_data[scale_cols])
X_train[scale_cols] = scaler.transform(X_train[scale_cols])
X_test[scale_cols]  = scaler.transform(X_test[scale_cols])

print("\n✅ Scaled snapshot features successfully.")
print(f"   Scaled columns: {len(scale_cols)} of {X_train.shape[1]} total features.")

# ==============================================================
# 5️⃣ Final Sanity Checks
# ==============================================================

print("\n✅ Feature Matrices Ready:")
print(f"   X_train: {X_train.shape}, y_train: {y_train.shape}")
print(f"   X_test : {X_test.shape}, y_test : {y_test.shape}")

print("\n🔒 Consistency Check: ✅ Same users used for all models (LSTM, RF, XGBoost).")


###Corrleation for snapshots

In [ ]:

# --- Snapshot correlation (XGBoost & RF) ---
print("📊 Correlation Matrix — Snapshot Features (XGBoost & RF)")

corr_snapshot = X_train.corr()

plt.figure(figsize=(12,8))
sns.heatmap(corr_snapshot, cmap='coolwarm', annot=False, fmt=".2f", linewidths=0.5)
plt.title("📊 Feature Correlation Heatmap — Snapshot Data (XGBoost & RF)")
plt.show()

# Optional: List highly correlated pairs



corr_pairs = (
    corr_snapshot.where(np.triu(np.ones(corr_snapshot.shape), k=1).astype(bool))
    .stack()
    .reset_index()
)
corr_pairs.columns = ["Feature1", "Feature2", "Correlation"]
high_corr_snapshot = corr_pairs[corr_pairs["Correlation"].abs() > correlation_threshold]
display(high_corr_snapshot)


###XGBoost model and training

In [ ]:

xgb_model  = XGBClassifier(
    n_estimators=300,
    learning_rate=0.05,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    n_jobs=-1,
    eval_metric='auc',
    scale_pos_weight=2.6,
    min_child_weight=1,
    gamma=0.1
    #--tree_method='gpu_hist',
    #--predictor='gpu_predictor'

)
print("🚀 Training XGBoost...")
xgb_model .fit(X_train, y_train)


###RF model and training

In [ ]:
# ✅ Train Random Forest in parallel
rf_model = RandomForestClassifier(
    n_estimators=300,
    max_depth=10,
    min_samples_split=5,
    min_samples_leaf=3,
    random_state=42,
    n_jobs=-1
)

print("🌲 Training Random Forest...")
rf_model.fit(X_train, y_train)





###Testing

In [ ]:

snapshot_indices = []
snapshot_metrics_xgb = []
snapshot_metrics_rf = []
recalls_xgb = []
recalls_rf = []
f1s_xgb, f1s_rf = [], []


for snap_idx, group in test_df.groupby('snapshot_index'):
    y_true = group['label']
    if y_true.nunique() < 2:
        continue

    X_snap = prepare_features(group)
    X_snap = X_snap.reindex(columns=X_train.columns, fill_value=0)

    # 🔹 XGBoost
    y_pred_xgb = xgb_model.predict_proba(X_snap)[:, 1]
    auc_xgb = roc_auc_score(y_true, y_pred_xgb)
    rec_xgb = recall_score(y_true, (y_pred_xgb > threshold).astype(int))
    f1_xgb  = f1_score(y_true, (y_pred_xgb > threshold).astype(int))


    # 🔹 Random Forest
    y_pred_rf = rf_model.predict_proba(X_snap)[:, 1]
    auc_rf = roc_auc_score(y_true, y_pred_rf)
    rec_rf = recall_score(y_true, (y_pred_rf > threshold).astype(int))
    f1_rf  = f1_score(y_true, (y_pred_rf > threshold).astype(int))


    # Append results
    snapshot_indices.append(snap_idx)
    snapshot_metrics_xgb.append(auc_xgb)
    snapshot_metrics_rf.append(auc_rf)
    recalls_xgb.append(rec_xgb)
    recalls_rf.append(rec_rf)
    f1s_xgb.append(f1_xgb)
    f1s_rf.append(f1_rf)


###Report

In [ ]:
# =====================================================
# ✅ Model Evaluations — Unified Format
# =====================================================

# Evaluate models using the unified evaluator
xgb_results = evaluate_global(xgb_model, X_test, y_test, model_name="XGBoost")
rf_results  = evaluate_global(rf_model,  X_test, y_test, model_name="Random Forest")

# =====================================================
# ✅ Global Model Comparison Summary
# =====================================================
summary = pd.DataFrame([xgb_results, rf_results])
print("\n📋 Global Model Comparison Summary:")
display(summary)



# =====================================================
# 📊 2. Model Comparison — XGBoost vs Random Forest
# =====================================================
plt.figure(figsize=(10, 6))

# --- AUC ---
plt.plot(snapshot_indices, snapshot_metrics_xgb, 'b-o', label='XGBoost AUC')
plt.plot(snapshot_indices, snapshot_metrics_rf, 'g--o', label='RandomForest AUC')

# --- Recall ---
plt.plot(snapshot_indices, recalls_xgb, color='orange', marker='s', linestyle='--', label='XGBoost Recall')
plt.plot(snapshot_indices, recalls_rf, color='red', marker='^', linestyle='--', label='RandomForest Recall')

# --- F1 ---
plt.plot(snapshot_indices, f1s_xgb, color='purple', marker='d', linestyle='--', label='XGBoost F1')
plt.plot(snapshot_indices, f1s_rf, color='brown', marker='x', linestyle='--', label='RandomForest F1')

plt.title('📈 Model Comparison: XGBoost vs Random Forest Over Time')
plt.xlabel('Snapshot Index')
plt.ylabel('Metric Value')
plt.legend()
plt.grid(True, linestyle='--', alpha=0.5)
plt.tight_layout()
plt.show()



### Feature Importance

In [ ]:
def plot_feature_importance(model, X_train, model_name="Model", top_n=20):
    """
    Plot feature importance for tree-based models (XGBoost, RandomForest).
    """


    # Handle model type
    if hasattr(model, "get_booster"):  # XGBoost
        importance = model.get_booster().get_score(importance_type='gain')
        fi = pd.DataFrame({
            'Feature': list(importance.keys()),
            'Importance': list(importance.values())
        })
    elif hasattr(model, "feature_importances_"):  # RandomForest
        fi = pd.DataFrame({
            'Feature': X_train.columns,
            'Importance': model.feature_importances_
        })
    else:
        raise ValueError(f"{model_name} does not support feature importance extraction.")

    # Sort and plot
    fi = fi.sort_values(by='Importance', ascending=False)
    display(fi.head(10))

    plt.figure(figsize=(10,6))
    plt.barh(fi['Feature'][:top_n][::-1], fi['Importance'][:top_n][::-1])
    plt.title(f'📊 {model_name} Feature Importance (Top {top_n})')
    plt.xlabel('Importance')
    plt.ylabel('Feature')
    plt.grid(alpha=0.4)
    plt.tight_layout()
    plt.show()

    return fi

fi_xgb = plot_feature_importance(xgb_model, X_train, "XGBoost")
fi_rf = plot_feature_importance(rf_model, X_train, "Random Forest")


##Advance ML

####Generate timeline

#####Load

In [ ]:
def load_raw_datasets(config):


    if "ML" in config and "Events" in config["ML"]:
        events_cfg = config["ML"]["Events"]
    else:
        events_cfg = config["Events"]

    base = events_cfg["base_path"]
    files = events_cfg["files"]

    # --- Load all datasets ---
    df_voc = pd.read_csv(os.path.join(base, files["voc"]))
    df_sms = pd.read_csv(os.path.join(base, files["sms"]))
    df_app = pd.read_csv(os.path.join(base, files["app"]))
    df_user = pd.read_csv(os.path.join(base, files["user"]))

    # --- Normalize timestamps and add source column ---
    for df, src in [(df_voc, "VOC"), (df_sms, "SMS"), (df_app, "APP"), (df_user, "USER")]:
        df["source"] = src
        ts_col = [c for c in df.columns if "time" in c.lower()][0]
        df.rename(columns={ts_col: "event_time"}, inplace=True)
        df["event_time"] = pd.to_datetime(df["event_time"], errors="coerce")

    print("✅ Raw datasets loaded and timestamp-normalized.")
    return df_voc, df_sms, df_app, df_user

df_voc, df_sms, df_app, df_user = load_raw_datasets(config)


#####Define user sequence

In [ ]:

def make_user_sequences(events, feature_cols=None, max_seq_len=100):
    """
    Build per-user sequences for LSTM models.
    Each user's events are sorted by time and padded/truncated to fixed length.

    Parameters
    ----------
    events : pd.DataFrame
        Combined event dataset (all sources).
    feature_cols : list or None
        List of numeric columns to include as features.
        If None, uses all numeric columns except 'label'.
    max_seq_len : int
        Sequence length to pad/truncate to.

    Returns
    -------
    X_seq : np.ndarray
        Array of shape (n_users, max_seq_len, n_features)
    y : np.ndarray
        Array of shape (n_users,)
    users : list
        List of user IDs
    """
    events = events.copy()
    users, X_seq, y = [], [], []

    # 🔹 Encode categorical 'source' column numerically
    le = LabelEncoder()
    events["source_id"] = le.fit_transform(events["source"].astype(str))

    # 🔹 Determine feature columns
    if feature_cols is None:
        feature_cols = events.select_dtypes(include=["number"]).columns.difference(["label"]).tolist()
    if "source_id" not in feature_cols:
        feature_cols.append("source_id")

    print(f"\n📦 Using {len(feature_cols)} features: {feature_cols}")


    # ✅ Build per-user sequences
    for user, df_u in events.groupby("phone_no_m"):
        df_u = df_u.sort_values("event_time")

        feats = df_u[feature_cols].to_numpy(dtype=float)

        # Pad or truncate
        if len(feats) < max_seq_len:
            feats = np.pad(feats, ((max_seq_len - len(feats), 0), (0, 0)))
        else:
            feats = feats[-max_seq_len:]

        # User label = any fraud event → fraud
        label = int(df_u["label"].max())

        X_seq.append(feats)
        y.append(label)
        users.append(user)

    print(f"\n✅ Created sequences for {len(users)} users")
    print(f"   Fraud users: {sum(y)} ({np.mean(y)*100:.2f}%)")
    print(f"   Normal users: {len(y) - sum(y)} ({(1 - np.mean(y))*100:.2f}%)")

    X_seq = np.array(X_seq)
    y = np.array(y)

    print(f"\n📐 Final tensor shape: X={X_seq.shape}, y={y.shape}")
    return X_seq, y, users


#####Build timeline (events)

In [ ]:
def merge_and_prepare_events(df_voc, df_sms, df_app, df_user):

    # --- 1️⃣ Normalize USER dataset ---
    if 'label' not in df_user.columns:
        raise KeyError("❌ 'label' column not found in user dataset")

    # Ensure numeric consistency
    df_user['label'] = df_user['label'].fillna(0).astype(int)
    df_user['idcard_cnt'] = df_user['idcard_cnt'].fillna(0).astype(float)
    df_user['arpu_value'] = df_user['arpu_value'].fillna(0).astype(float)

    # --- 2️⃣ Extract static info for merging (label + sim count only) ---
    static_user_info = df_user.groupby("phone_no_m", as_index=False)[["label", "idcard_cnt"]].max()

    # --- 3️⃣ Merge static info into other event tables ---
    df_voc = df_voc.merge(static_user_info, on="phone_no_m", how="left")
    df_sms = df_sms.merge(static_user_info, on="phone_no_m", how="left")
    df_app = df_app.merge(static_user_info, on="phone_no_m", how="left")


    # --- 4️⃣ Combine all transactional event sources ---
    # include df_user itself since arpu_value is event-like
    events = pd.concat([df_voc, df_sms, df_app, df_user], ignore_index=True)
    # ✅ Keep only transactional events (VOC + SMS)
    #Drop app and user fee
    #events = pd.concat([df_voc, df_sms], ignore_index=True)

    # --- 5️⃣ Fill and order ---
    events["label"] = events["label"].fillna(0).astype(int)
    events["event_time"] = pd.to_datetime(events["event_time"], errors="coerce")
    events = events.sort_values(["phone_no_m", "event_time"]).reset_index(drop=True)

    # --- 6️⃣ Summary ---
    print("\n🔎 Feature Summary per Source:")
    for src, df in [("VOC", df_voc), ("SMS", df_sms), ("APP", df_app), ("USER", df_user)]:
        print(f"\n📂 Source: {src}")
        print(f"   Events: {len(df):,}")
        print(f"   Users : {df['phone_no_m'].nunique():,}")
        print(f"   Columns ({len(df.columns)}): {', '.join(df.columns)}")

    print("\n📊 Combined Dataset Summary:")
    print(f"   Total events: {len(events):,}")
    print(f"   Unique users: {events['phone_no_m'].nunique():,}")
    print(f"   Fraud ratio: {events['label'].mean()*100:.2f}%")

    return events

events = merge_and_prepare_events(df_voc, df_sms, df_app, df_user)


#####Split data based on users (fraud, not fraud)

In [ ]:


# ======================================
# 0️⃣ Clean Numeric Columns
# ======================================
events = events.copy()
numeric_cols = events.select_dtypes(include=["number"]).columns.difference(["label"])

# Replace NaN with 0 for numeric fields (avoids scaling issues)
events[numeric_cols] = events[numeric_cols].fillna(0)

print(f"\n📊 Numeric columns to scale ({len(numeric_cols)}): {numeric_cols.tolist()}")

# ======================================
# 1️⃣ Scale Numeric Features
# ======================================
scaler_seq = StandardScaler()
events[numeric_cols] = scaler_seq.fit_transform(events[numeric_cols])
print(f"📏 Scaled {len(numeric_cols)} numeric columns for event-level modeling.")

# ======================================
# 2️⃣ Create Train/Test User Split (if not exists)
# ======================================
split_dir = "splits"
train_split_file = f"{split_dir}/train_users.csv"
test_split_file = f"{split_dir}/test_users.csv"

os.makedirs(split_dir, exist_ok=True)

if os.path.exists(train_split_file) and os.path.exists(test_split_file):
    print("📂 Using existing user split from file...")
    train_users = set(pd.read_csv(train_split_file)["phone_no_m"])
    test_users  = set(pd.read_csv(test_split_file)["phone_no_m"])
else:
    print("🆕 Creating new unified user split (for LSTM)...")

    # One label per user
    user_labels = events.groupby("phone_no_m")["label"].max()
    fraud_users = user_labels[user_labels == 1].index
    normal_users = user_labels[user_labels == 0].index

    fraud_train, fraud_test = train_test_split(fraud_users, test_size=0.2, random_state=42)
    normal_train, normal_test = train_test_split(normal_users, test_size=0.2, random_state=42)

    train_users = set(fraud_train) | set(normal_train)
    test_users  = set(fraud_test)  | set(normal_test)

    pd.DataFrame({"phone_no_m": sorted(train_users)}).to_csv(train_split_file, index=False)
    pd.DataFrame({"phone_no_m": sorted(test_users)}).to_csv(test_split_file, index=False)
    print(f"✅ Saved user split to '{split_dir}/'")

print(f"✅ Train users: {len(train_users)} | Test users: {len(test_users)}")

# ======================================
# 3️⃣ Apply Split to Events
# ======================================
train_events = events[events["phone_no_m"].isin(train_users)]
test_events  = events[events["phone_no_m"].isin(test_users)]

# --- add time gap, scaled featur ---
for name, df in [('train_events', train_events), ('test_events', test_events)]:
    df = df.copy()  # avoid SettingWithCopyWarning
    df['event_time'] = pd.to_datetime(df['event_time'])
    df.sort_values(['phone_no_m', 'event_time'], inplace=True)
    df['dt_hours'] = df.groupby('phone_no_m')['event_time'].diff().dt.total_seconds() / 3600
    df['dt_hours'] = df['dt_hours'].fillna(0)
    df['dt_hours'] = np.log1p(df['dt_hours'])  # normalize gaps
    if name == 'train_events':
        train_events = df
    else:
        test_events = df


# Sanity checks
assert len(set(train_events["phone_no_m"]) & set(test_events["phone_no_m"])) == 0, "❌ User leakage detected!"
assert train_events["label"].nunique() == 2, "❌ Training set must contain both classes"
assert test_events["label"].nunique() == 2, "❌ Test set must contain both classes"

# ======================================
# 4️⃣ Create Sequences (using multiple features)
# ======================================
numeric_features = [c for c in numeric_cols if c not in ["label"]]  # exclude label
if 'dt_hours' in train_events.columns:
    numeric_features.append('dt_hours')
print(f"\n📦 Features used for sequences: {numeric_features}")

X_train, y_train, users_train = make_user_sequences(train_events, feature_cols=numeric_features, max_seq_len=max_seq_len)
X_test, y_test, users_test = make_user_sequences(test_events, feature_cols=numeric_features, max_seq_len=max_seq_len)

print("\n✅ Sequence Summary (per-user sequences):")
print(f"   X_train: {X_train.shape} | Fraud ratio: {np.mean(y_train)*100:.2f}%")
print(f"   X_test : {X_test.shape} | Fraud ratio: {np.mean(y_test)*100:.2f}%")

# ======================================
# 5️⃣ Consistency Check
# ======================================
rf_train = set(pd.read_csv(train_split_file)["phone_no_m"])
rf_test  = set(pd.read_csv(test_split_file)["phone_no_m"])
assert rf_train == train_users, "❌ Train user mismatch between LSTM and RF/XGB!"
assert rf_test  == test_users,  "❌ Test user mismatch between LSTM and RF/XGB!"
print("\n🔒 Consistency Check: ✅ Same users used for all models (LSTM, RF, XGBoost).")


#####Correlation raw data

In [ ]:
# --- LSTM correlation ---
print("📊 Correlation Matrix — Raw Event Features (LSTM)")

corr_lstm = pd.DataFrame(events[numeric_cols]).corr()

plt.figure(figsize=(12,8))
sns.heatmap(corr_lstm, cmap='coolwarm', annot=False, fmt=".2f", linewidths=0.5)
plt.title("📊 Feature Correlation Heatmap — Raw Event Data (LSTM)")
plt.show()

# Optional: Highly correlated pairs

corr_pairs_lstm = (
    corr_lstm.where(np.triu(np.ones(corr_lstm.shape), k=1).astype(bool))
    .stack()
    .reset_index()
)
corr_pairs_lstm.columns = ["Feature1", "Feature2", "Correlation"]
high_corr_lstm = corr_pairs_lstm[corr_pairs_lstm["Correlation"].abs() > correlation_threshold]
display(high_corr_lstm)


#####Class weights

In [ ]:

weights = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_train),
    y=y_train
)
class_weights = dict(enumerate(weights))
print(class_weights)

###RNN - LSTM

#####F1 Calculation

In [ ]:

def f1_metric(y_true, y_pred):
    # Convert both tensors to float32 before math operations
    y_true = K.cast(y_true, 'float32')
    y_pred = K.cast(K.round(y_pred), 'float32')

    tp = K.sum(y_true * y_pred)
    fp = K.sum((1 - y_true) * y_pred)
    fn = K.sum(y_true * (1 - y_pred))

    precision = tp / (tp + fp + K.epsilon())
    recall = tp / (tp + fn + K.epsilon())
    return 2 * ((precision * recall) / (precision + recall + K.epsilon()))


#####Model and training

In [ ]:


# ======================================
# 4️⃣ Build and train LSTM model
# ======================================

lstm_model = Sequential([
    Masking(mask_value=0.0, input_shape=(max_seq_len, X_train.shape[2])),
    LSTM(128, return_sequences=False, use_cudnn=False),
    Dropout(dropout),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')
])


lstm_model.compile(
    loss='binary_crossentropy',
    optimizer=Adam(1e-4),
    metrics=['AUC', 'Recall',f1_metric]
)

print("🚀 Training LSTM...")
lstm_history = lstm_model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=epochs , batch_size=batch_size,
    class_weight=class_weights

)


##### Testing

In [ ]:

# Predict probabilities and labels
y_pred_prob = lstm_model.predict(X_test).ravel()
y_pred = (y_pred_prob > threshold).astype(int)


#####Report

In [ ]:
# =====================================================
# ✅ LSTM Evaluation — Unified Format
# =====================================================

# Evaluate LSTM model using the unified evaluator
lstm_results = evaluate_global(lstm_model, X_test, y_test, model_name="LSTM")

# Add to global summary
summary = pd.concat([summary, pd.DataFrame([lstm_results])], ignore_index=True)

# Display updated summary
print("\n📋 Updated Model Comparison Summary:")
display(summary)

In [ ]:
# =====================================================
# 📈 1. LSTM Training Metrics (AUC + F1)
# =====================================================
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# --- AUC Plot ---
axes[0].plot(lstm_history.history['AUC'], label='Train AUC', color='blue')
axes[0].plot(lstm_history.history['val_AUC'], label='Val AUC', color='orange')
axes[0].set_title('LSTM — AUC Over Epochs')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('AUC')
axes[0].legend()
axes[0].grid(True, alpha=0.4)

# --- F1 Plot ---
axes[1].plot(lstm_history.history['f1_metric'], label='Train F1', color='purple')
axes[1].plot(lstm_history.history['val_f1_metric'], label='Val F1', color='magenta')
axes[1].set_title('LSTM — F1 Over Epochs')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('F1 Score')
axes[1].legend()
axes[1].grid(True, alpha=0.4)

plt.suptitle('📊 LSTM Training Performance', fontsize=14, y=1.05)
plt.tight_layout()
plt.show()


###Transformer

####Build sequence

In [ ]:
# ==============================================================
# NEW: Transformer helpers (no changes to existing LSTM path)
# ==============================================================


def build_tf_sequences(events, feature_cols=None, max_seq_len=100):
    """
    Build Transformer-ready sequences with:
      - X_seq:      [n_users, T, F]   numeric features
      - DT_seq:     [n_users, T, 1]   delta-time (hours) feature
      - PAD_mask:   [n_users, T]      True where PAD
      - y_user:     [n_users]         user labels
      - users:      list              phone_no_m
    NOTE: This is separate from make_user_sequences(); it does not replace it.
    """
    events = events.copy()
    users, X_seq, DT_seq, PAD_mask, y_user = [], [], [], [], []
    D_seq = []

    # Ensure numeric source_id exists (you already add it elsewhere)
    if "source_id" not in events.columns:

        le = LabelEncoder()
        events["source_id"] = le.fit_transform(events["source"].astype(str))

    # Default: all numeric except label
    if feature_cols is None:
        feature_cols = events.select_dtypes(include=["number"]).columns.difference(["label"]).tolist()
    if "source_id" not in feature_cols:
        feature_cols.append("source_id")

    for user, df_u in events.groupby("phone_no_m"):
        df_u = df_u.sort_values("event_time")

        # delta-time in hours between events (0 for first)
        #dt_hours = df_u["event_time"].diff().dt.total_seconds().fillna(0) / 3600.0
        #dt_hours = dt_hours.to_numpy(dtype=float).reshape(-1, 1)
        # ✅ use already-computed normalized dt_hours
        dt_hours = df_u["dt_hours"].to_numpy(dtype=float).reshape(-1, 1)



        feats = df_u[feature_cols].to_numpy(dtype=float)
        L = len(feats)

        # Build padding mask: True where PAD (we left-pad to keep most recent)
        if L < max_seq_len:
            pad_len = max_seq_len - L
            feats   = np.pad(feats,   ((pad_len, 0), (0, 0)))
            dt_hours= np.pad(dt_hours,((pad_len, 0), (0, 0)))
            pad_mask = np.zeros((max_seq_len,), dtype=bool)
            pad_mask[:pad_len] = True
        else:
            feats    = feats[-max_seq_len:]
            dt_hours = dt_hours[-max_seq_len:]
            pad_mask = np.zeros((max_seq_len,), dtype=bool)

        label = int(df_u["label"].max())

        X_seq.append(feats)
        DT_seq.append(dt_hours)
        PAD_mask.append(pad_mask)
        y_user.append(label)
        users.append(user)
        D_seq.append(df_u['dt_hours'].values[-max_seq_len:])



    X_seq   = np.stack(X_seq)
    DT_seq  = np.stack(DT_seq)
    PAD_mask= np.stack(PAD_mask)
    y_user  = np.array(y_user, dtype=int)
    return X_seq, DT_seq, PAD_mask, y_user, users


print("\n🧪 [Transformer] Building sequences...")
max_seq_len_tf = limit

# Build Transformer-ready sequences (no leakage; uses your existing train/test splits)
Xtr_raw, DTr_raw, Mtr, ytr, users_tr = build_tf_sequences(train_events, max_seq_len=max_seq_len_tf)
Xte_raw, DTe_raw, Mte, yte, users_te = build_tf_sequences(test_events,  max_seq_len=max_seq_len_tf)

####Scale, prepare  and select device

In [ ]:
# Scale numeric features (fit only on training)

num_features = Xtr_raw.shape[2]
scaler_tf = StandardScaler().fit(Xtr_raw.reshape(-1, num_features))
Xtr = scaler_tf.transform(Xtr_raw.reshape(-1, num_features)).reshape(Xtr_raw.shape)
Xte = scaler_tf.transform(Xte_raw.reshape(-1, num_features)).reshape(Xte_raw.shape)

# Convert to tensors

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
display(device)


Xtr_t = torch.tensor(Xtr, dtype=torch.float32).to(device)
Xte_t = torch.tensor(Xte, dtype=torch.float32).to(device)
DTr_t = torch.tensor(DTr_raw, dtype=torch.float32).to(device)
DTe_t = torch.tensor(DTe_raw, dtype=torch.float32).to(device)
Mtr_t = torch.tensor(Mtr, dtype=torch.bool).to(device)
Mte_t = torch.tensor(Mte, dtype=torch.bool).to(device)
ytr_t = torch.tensor(ytr, dtype=torch.float32).to(device)
yte_t = torch.tensor(yte, dtype=torch.float32).to(device)

# DataLoaders
train_ds = TensorDataset(Xtr_t, DTr_t, Mtr_t, ytr_t)
test_ds  = TensorDataset(Xte_t, DTe_t, Mte_t, yte_t)
train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
test_loader  = DataLoader(test_ds,  batch_size=batch_size, shuffle=False)




##### Custom Transformer

###### Model

In [ ]:

# ----------------- PyTorch Transformer model -----------------



class TransformerFraud(nn.Module):
    def __init__(self, input_dim, d_model=128, n_heads=4, n_layers=2, time_k=7,dropout=0.0):
        super().__init__()
        #self.time2vec = TimeEncoder(mode=time_mode, k=time_k)
#        self.proj = nn.Linear(input_dim + (1 + time_k), d_model)
        self.proj = nn.Linear(input_dim, d_model)  # match feature count without time2vec


        enc_layer = nn.TransformerEncoderLayer(
            d_model=d_model, nhead=n_heads, batch_first=True,dropout=dropout
        )
        self.encoder = nn.TransformerEncoder(enc_layer, num_layers=n_layers)
        self.cls = nn.Sequential(
            nn.Linear(d_model, 64), nn.ReLU(), nn.Dropout(dropout),nn.Linear(64, 1)
        )

    def forward(self, x, dt_hours, pad_mask=None):  # x:[B,T,F], dt:[B,T,1], pad_mask:[B,T] (True=PAD)
        #x = torch.cat([x, self.time2vec(dt_hours)], dim=-1)  # augment with time code
        h = self.proj(x)
        h = self.encoder(h, src_key_padding_mask=pad_mask)   # mask: True entries ignored
        # mask-aware mean pool
        if pad_mask is not None:
            keep = (~pad_mask).unsqueeze(-1)                 # [B,T,1]
            denom = keep.sum(dim=1).clamp(min=1)
            h = (h * keep).sum(dim=1) / denom
        else:
            h = h.mean(dim=1)
        logit = self.cls(h).squeeze(-1)
        return torch.sigmoid(logit)

# Instantiate Transformer model
model_tf = TransformerFraud(input_dim=Xtr.shape[2], d_model=128, n_heads=4, n_layers=2, time_k=7,dropout=dropout).to(device)
opt = torch.optim.Adam(model_tf.parameters(), lr=1e-4)
bce = torch.nn.BCELoss()#Binary Cross-Entropy Loss









###### Training

In [ ]:

# Training loop
for ep in range(1, epochs + 1):
    model_tf.train()
    loss_sum = 0.0
    for xb, dtb, mb, yb in train_loader:
        pred = model_tf(xb, dtb, pad_mask=mb)
        loss = bce(pred, yb)
        weights = torch.tensor([class_weights[int(label)] for label in yb.cpu().numpy()]).to(device)
        loss = (loss * weights).mean()
        opt.zero_grad()
        loss.backward()
        opt.step()
        loss_sum += loss.item() * len(xb)
    print(f"[Transformer][Epoch {ep}] train_loss={loss_sum / len(train_ds):.4f}")

# Evaluation


model_tf.eval()
with torch.no_grad():
    preds = []
    for xb, dtb, mb, yb in test_loader:
        p = model_tf(xb, dtb, pad_mask=mb)
        preds.append(p.detach().cpu().numpy())
    p_te = np.concatenate(preds)

auc  = roc_auc_score(yte, p_te)
ap   = average_precision_score(yte, p_te)
print(f"[Transformer] Test ROC-AUC: {auc:.4f} | PR-AUC: {ap:.4f}")

tf_user_scores = pd.DataFrame({"phone_no_m": users_te, "p_tf": p_te, "y": yte})
print("✅ [Transformer] Inference complete. Sample:")
print(tf_user_scores.head())

###### Tansformer Report

In [ ]:
# ==============================================================
# ADDITION: Extended Transformer evaluation metrics + combined plot
# ==============================================================


# Convert probabilities to binary predictions
pred_label = (p_te >= threshold).astype(int)

# Compute additional metrics
precision = precision_score(yte, pred_label)
recall = recall_score(yte, pred_label)
f1 = f1_score(yte, pred_label)

print(f"[Transformer] Precision: {precision:.4f} | Recall: {recall:.4f} | F1-score: {f1:.4f}")

# Curves
fpr, tpr, _ = roc_curve(yte, p_te)
prec, rec, _ = precision_recall_curve(yte, p_te)

# Combined ROC and PR plot
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, label=f"ROC Curve (AUC={auc:.3f})", color='blue')
plt.plot(rec, prec, label=f"Recall Curve (AUC={ap:.3f})", color='orange')
plt.plot([0, 1], [0, 1], "k--", alpha=0.3)
plt.xlabel("False Positive Rate / Recall")
plt.ylabel("True Positive Rate / Precision")
plt.title("TransformerFraud: ROC and Precision-Recall Curves")
plt.legend(loc="lower right")
plt.grid(True, linestyle='--', alpha=0.5)
plt.tight_layout()
plt.show()
# ==============================================================
# SYNCED SUMMARY ENTRY — same structure as LSTM, RF, XGB
# ==============================================================

# Drop any old TransformerFraud entry
summary = summary[summary["Model"] != "TransformerFraud"]

# Create results dictionary (consistent with evaluate_lstm_global output)
transformer_results = {
    "Model": "TransformerFraud",
    "AUC": round(auc, 4),
    "Recall": round(recall, 4),
    "Precision": round(precision, 4),
    "F1": round(f1, 4)
}

# Add PR-AUC only if you want to extend (not required for sync)
#transformer_results["PR-AUC"] = round(ap, 4)

# Standardize columns
expected_cols = ["Model", "AUC", "Recall", "Precision", "F1"]
if "summary" not in locals():
    summary = pd.DataFrame(columns=expected_cols)

# Concatenate and reindex
summary = pd.concat([summary, pd.DataFrame([transformer_results])], ignore_index=True)
summary = summary.reindex(columns=expected_cols)

display(summary)



#Pretrained

##Informer Pretrained

#### Install

In [ ]:
# Clean up any old copies
!rm -rf Informer2020

# Clone the official repo
!git clone https://github.com/zhouhaoyi/Informer2020.git

# Go inside the repo
%cd Informer2020
import sys
sys.path.append('/content/Informer2020')  # adjust path if needed
from exp.exp_informer import Exp_Informer
from models.model import Informer

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"✅ Using device: {device}")
print("✅ Informer2020 imported successfully!")


#### Zero shot Model

In [ ]:
# # =====================================================
# # ✅ 1. Imports & Device
# # =====================================================
# import torch
# import torch.nn as nn
# from torch.cuda.amp import autocast
# from sklearn.covariance import EmpiricalCovariance
# from sklearn.metrics import roc_auc_score, average_precision_score, precision_score, recall_score, f1_score
# import numpy as np
# import pandas as pd
# import matplotlib.pyplot as plt



In [ ]:
class InformerWrapper(nn.Module):
    """
    Informer backbone used as a zero-shot embedding generator.
    Matches TransformerFraud structure where possible.
    """
    def __init__(self, input_dim, num_labels=2, dropout=0.0,seq_len=96, freeze_backbone=True):
        super().__init__()
        self.seq_len = seq_len
        seq_len = self.seq_len
        self.backbone = Informer(
            enc_in=input_dim,
            dec_in=input_dim,
            c_out=input_dim,
            seq_len=96,           # keep consistent with Transformer input shape
            label_len=48,         # Informer requirement
            out_len=24,           # Informer requirement
            factor=3,             # Informer-specific; can’t change
            d_model=128,          # ✅ matched to TransformerFraud
            n_heads=4,            # ✅ matched
            e_layers=2,           # ✅ matched
            d_layers=1,           # ✅ minimal, required
            d_ff=512,             # ✅ light FFN, keeps efficiency
            dropout=dropout,      # ✅ match TF
            attn='prob',
            embed='timeF',
            freq='h',
            activation='gelu',
            output_attention=False,
            distil=True,
            mix=True,
            device=device
        )

        # ✅ Zero-shot: freeze backbone
        if freeze_backbone:
            for p in self.backbone.parameters():
                p.requires_grad = False


        #This head converts the Informer’s pooled output into a single logit (fraud score)
        self.proj = nn.Sequential(
            nn.Linear(input_dim, 64),   # ✅ match TF hidden size
            nn.ReLU(),
            nn.Dropout(dropout),        # ✅ same regularization
            nn.Linear(64, 1)            # ✅ single logit like TransformerFraud
        )

    def _make_time_mark(self, x):
        """Generate dummy time encodings."""
        B, T, _ = x.shape
        t = torch.linspace(0, 1, T, device=x.device).unsqueeze(0).repeat(B, 1)
        mark = torch.stack([t, t, t, t], dim=-1)
        return mark

    def forward(self, x):
        """
        Match Transformer input handling: [B, T, F].
        """
        B, T, F = x.shape

        # ✅ Keep consistent: pad/trim to seq_len (same approach TF uses with positional mask)
        seq_len = self.seq_len


        if T < seq_len:
            pad = torch.zeros(B, seq_len - T, F, device=x.device)
            x_enc = torch.cat([pad, x], dim=1)
        else:
            x_enc = x[:, -seq_len:, :]

        # Informer needs decoder placeholders
        x_dec = torch.zeros(B, 48 + 24, F, device=x.device)
        x_mark_enc = self._make_time_mark(x_enc)
        x_mark_dec = self._make_time_mark(x_dec)

        # ✅ Forward pass identical to Informer paper
        out = self.backbone(x_enc, x_mark_enc, x_dec, x_mark_dec)
        if isinstance(out, tuple):
            out = out[0]

        # ✅ Average pool like TransformerFraud mean pooling
        pooled = out.mean(dim=1)
        logits = self.proj(pooled)
        return logits

@torch.no_grad()
def _embed_logits(model, X, batch_size=8):
    model.eval()
    device = next(model.parameters()).device
    outputs = []

    for i in range(0, len(X), batch_size):
        batch = X[i:i+batch_size].to(device)
        with autocast():
            logits = model(batch)
        outputs.append(logits.cpu())
        torch.cuda.empty_cache()

    return torch.cat(outputs, dim=0).numpy()


def get_safe_batch_size(X, model, max_batch=64):
    for b in [max_batch, 32, 16, 8, 4, 2, 1]:
        try:
            _ = model(X[:b].to(next(model.parameters()).device))
            return b
        except RuntimeError as e:
            if "out of memory" in str(e):
                torch.cuda.empty_cache()
                continue
            else:
                raise e
    return 1

###Zero-Shot Evaluation

In [ ]:


# =====================================================
# ✅ 5. Zero-Shot Evaluation
# =====================================================
seq_len = limit                      # ✅ same window size as LSTM/Transformer
input_dim = Xtr_t.shape[2]
dropout_val = dropout                # ✅ same dropout variable as others

informer_zero = InformerWrapper(
    input_dim=input_dim,
    seq_len=seq_len,
    dropout=dropout_val,
    freeze_backbone=True
).to(device)
informer_zero.eval()

safe_b = get_safe_batch_size(Xtr_t[:128], informer_zero, max_batch=32)
print(f"✅ Safe batch size: {safe_b}")

# ---- Normal training data (no frauds) ----
train_normal = (ytr_t.cpu().numpy() == 0)
emb_norm = _embed_logits(informer_zero, Xtr_t[train_normal], batch_size=safe_b)

# ---- Fit covariance on normal embeddings ----
cov = EmpiricalCovariance().fit(emb_norm)
mu, VI = cov.location_, cov.precision_

def mahalanobis_batch(X, mean, VI):
    diff = X - mean
    return np.sqrt(np.einsum("ij,jk,ik->i", diff, VI, diff))

# ---- Test embeddings ----
emb_test = _embed_logits(informer_zero, Xte_t, batch_size=safe_b)
scores = mahalanobis_batch(emb_test, mu, VI)
scores = (scores - scores.min()) / (np.ptp(scores) + 1e-8)

# ---- Threshold (reuse LSTM/TF logic) ----
threshold = np.percentile(scores, 95)     # or your LSTM best-F1 threshold

# ---- Metrics ----
y_true = yte_t.cpu().numpy()
pred_label = (scores >= threshold).astype(int)

precision = precision_score(y_true, pred_label, zero_division=0)
recall = recall_score(y_true, pred_label, zero_division=0)
f1 = f1_score(y_true, pred_label, zero_division=0)
auc_val = roc_auc_score(y_true, scores)
ap_val = average_precision_score(y_true, scores)

print("==============================================================")
print(f"[ZERO-SHOT InformerFraud] Precision: {precision:.4f} | Recall: {recall:.4f} | F1: {f1:.4f}")
print(f"AUC: {auc_val:.4f} | PR-AUC: {ap_val:.4f}")
print("==============================================================")


# =====================================================
# ✅ 6. ROC & PR Curves
# =====================================================
fpr, tpr, _ = roc_curve(y_true, scores)
prec, rec, _ = precision_recall_curve(y_true, scores)

plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, label=f"ROC (AUC={auc_val:.3f})", color='blue')
plt.plot(rec, prec, label=f"PR (AUC={ap_val:.3f})", color='orange')
plt.plot([0, 1], [0, 1], "k--", alpha=0.3)
plt.xlabel("False Positive Rate / Recall")
plt.ylabel("True Positive Rate / Precision")
plt.title("Informer Zero-Shot: ROC & PR Curves")
plt.legend(loc="lower right")
plt.grid(True, linestyle='--', alpha=0.5)
plt.tight_layout()
plt.show()


# =====================================================
# ✅ 7. Append to Summary (same as LSTM / TF)
# =====================================================
if "summary" in locals():
    summary = summary[summary["Model"] != "InformerZeroShot"]
else:
    summary = pd.DataFrame(columns=["Model", "AUC", "Recall", "Precision", "F1"])

informer_results = {
    "Model": "InformerZeroShot",
    "AUC": round(auc_val, 4),
    "Recall": round(recall, 4),
    "Precision": round(precision, 4),
    "F1": round(f1, 4)
}

summary = pd.concat([summary, pd.DataFrame([informer_results])], ignore_index=True)
summary = summary.reindex(columns=["Model", "AUC", "Recall", "Precision", "F1"])

display(summary)


####Testing

####test

In [ ]:
import torch
import numpy as np
from transformers import InformerModel

# ==========================================================
# 1️⃣ Load pretrained model
# ==========================================================
model_name = "huggingface/informer-tourism-monthly"
print(f"🔄 Loading {model_name} ...")

model = InformerModel.from_pretrained(model_name)
config = model.config
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

print(f"✅ Using device: {device}")
print(f"✅ Model type: {type(model)}")

# ==========================================================
# 2️⃣ Stats
# ==========================================================
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"\n✅ Total parameters: {total_params:,}")
print(f"✅ Trainable parameters: {trainable_params:,}")

means = [p.data.mean().item() for p in model.parameters()]
stds = [p.data.std().item() for p in model.parameters()]
print(f"🔍 Mean(mean): {np.mean(means):.6f} | Mean(std): {np.mean(stds):.6f}")
print("✅ Weights distribution indicates pretrained values.\n")

# ==========================================================
# 3️⃣ Get configuration details
# ==========================================================
input_dim = getattr(config, "input_size", 1)
context_len = getattr(config, "context_length", 96)
pred_len = getattr(config, "prediction_length", 24)
lags = getattr(config, "lags_sequence", list(range(1, 38)))
max_lag = max(lags)
past_seq_len = context_len + max_lag

# The model internally uses about 21 time features in this checkpoint
num_time_features = getattr(config, "num_time_features", 21)

print(f"✅ Detected input_dim={input_dim}, context_len={context_len}, "
      f"pred_len={pred_len}, num_time_features={num_time_features}, max_lag={max_lag}")
print(f"✅ Past sequence length adjusted to {past_seq_len}\n")

# ==========================================================
# 4️⃣ Create valid dummy inputs
# ==========================================================
past_values = torch.randn(1, past_seq_len, input_dim).to(device)
past_time_features = torch.randn(1, past_seq_len, num_time_features).to(device)
past_observed_mask = torch.ones(1, past_seq_len).to(device)
future_time_features = torch.randn(1, pred_len, num_time_features).to(device)

# ==========================================================
# 5️⃣ Forward pass
# ==========================================================
with torch.no_grad():
    output = model(
        past_values=past_values,
        past_time_features=past_time_features,
        past_observed_mask=past_observed_mask,
        future_time_features=future_time_features,
    )

if hasattr(output, "last_hidden_state"):
    print(f"✅ Forward OK — output shape: {tuple(output.last_hidden_state.shape)}")
else:
    print(f"✅ Forward OK — tensor output: {type(output)}")

# ==========================================================
# 6️⃣ Determinism test
# ==========================================================
with torch.no_grad():
    o1 = model(
        past_values=past_values,
        past_time_features=past_time_features,
        past_observed_mask=past_observed_mask,
        future_time_features=future_time_features,
    ).last_hidden_state
    o2 = model(
        past_values=past_values,
        past_time_features=past_time_features,
        past_observed_mask=past_observed_mask,
        future_time_features=future_time_features,
    ).last_hidden_state

print(f"✅ Deterministic inference: {torch.allclose(o1, o2)}")
print("\n🎯 ✅ Model functional — pretrained weights loaded, dimensions aligned, forward verified.")


#freeze

In [ ]:
%pip freeze > "{project_path}requirement/freez/EnhancedPretraindMLModleAdvance-lock.txt"